In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
csv_path = "cities.csv"
ciudades = pd.read_csv(csv_path)
ciudades.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,lat,lon,Max Temp,Wind Speed
0,0,Luderitz,0,NaN,1579881280,50,-26.65,15.16,26.00,6.20
1,1,Butaritari,100,KI,1579881263,76,3.07,172.79,28.77,3.97
2,2,Klaksvik,100,FO,1579881289,93,62.23,-6.59,3.00,4.60
3,3,East London,68,ZA,1579881261,78,-33.02,27.91,24.00,6.70
4,4,Punta Arenas,90,CL,1579881258,87,-53.15,-70.92,10.00,5.10


In [3]:
# Store latitude and longitude in locations
locations = ciudades[["lat", "lon"]]

# Fill NaN values and convert to float
rating = ciudades["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
escogidos =ciudades.loc[ciudades["Max Temp"] <= 26, :]
escogidos =escogidos.loc[escogidos["Max Temp"] >= 22, :]
escogidos =escogidos.loc[escogidos["Cloudiness"] == 0, :]
escogidos =escogidos.loc[escogidos["Wind Speed"] <= 16, :]

In [6]:


# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "Hotels",
    "key": g_key
}

escogidos["Hotel Name"]=""
escogidos["Hotel Rating"]=""


# Use the lat/lng we recovered to identify airports
for index,citi in escogidos.iterrows():
   
    # get lat, lng from df
    lat = citi["lat"]
    lng = citi["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
   
    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        escogidos.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        escogidos.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
        
      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
hoteles1=escogidos[["Hotel Name","City","Country","Hotel Rating","lat","lon"]]
hoteles1

,Hotel Name,City,Country,Hotel Rating,lat,lon
0,Element Riders Backpackers,Luderitz,NaN,4.2,-26.65,15.16
14,Pension Bianca & Benoit,Rikitea,PF,4.2,-23.12,-134.97
21,Premier Hotel The Richards,Richards Bay,ZA,4,-28.78,32.04
41,Rivero Hotel Chuí,Chui,UY,3.5,-33.70,-53.46
58,A Piece Of Life,Bredasdorp,ZA,5,-34.53,20.04
73,Hotel Buongiorno,Erechim,BR,3.4,-27.63,-52.27
90,Hotel Plaza Lebu,Lebu,CL,4.3,-37.62,-73.65
258,De Duine Hotel,Henties Bay,NaN,4,-22.12,14.28
316,Al Salam Rotana,Khartoum,SD,4.4,15.55,32.53
331,Enjoy Coquimbo Hotel de la Bahía,Coquimbo,CL,4.5,-29.95,-71.34


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteles1.iterrows()]
locations = hoteles1[["lat", "lon"]]
hotel_layer = gmaps.marker_layer(
    locations ,info_box_content=hotel_info)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))